In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import numpy as np
import streamlit as st
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns 

def custom_round(x, base):
    return base * round(float(x) / base)

def check_data(df):
    for col in range(0, len(df.columns)):
        print(df.columns[col])
        print("_______________________")
        print(
            df[df.columns[col]].value_counts(dropna=False)
        )  # zmienne maja rozne poczatki, co ciekawe - nulli nie wykrywa dla niektorych danych

list_messy_data_histograms = []
list_napctg = []

messy_data = pd.read_csv("C:\DAscripts\messy_data.csv")

messy_data.columns = messy_data.columns.str.strip()
messy_data.replace(" ", np.nan, inplace=True)

na_count = messy_data.isna().to_numpy().sum()
na_pct = na_count / (messy_data.shape[0] * messy_data.shape[1])

messy_data = messy_data.convert_dtypes()
messy_data.iloc[:, 1:4] = messy_data.iloc[:, 1:4].apply(
    lambda x: x.astype(str).str.lower().astype("category")
)
messy_data.iloc[:, 4:8] = messy_data.iloc[:, 4:8].apply(lambda x: x.astype(float))
messy_data.iloc[:, 8:10] = messy_data.iloc[:, 8:10].apply(lambda x: x.astype(float))

##################### clean the data ####################################

import statsmodels.formula.api as smf
import seaborn as sns
import random
from scipy.stats.mstats import winsorize 

model_data = messy_data.copy()
model_data['color'] = np.where(model_data['color'].str.contains('colorless'), random.choice(['d','e','f']), model_data['color'])

imputer = IterativeImputer(random_state = 42, initial_strategy = 'median', sample_posterior = True,
                          imputation_order = 'ascending', min_value = 0)
imputed = imputer.fit_transform(model_data.iloc[:,[0,4,5,6,7,8,9]])
ready_data = pd.DataFrame(imputed)
ready_data.head()
model_data.iloc[:,[0,4,5,6,7,8,9]] = ready_data

model_data['depth'] = model_data['depth'].round(1)
model_data.iloc[:, model_data.columns.str.contains("dimension")] = model_data.iloc[:, model_data.columns.str.contains("dimension")].round(2)
model_data['table'] = model_data['table'].astype('int')
model_data['carat'] = model_data['carat'].apply(lambda x: custom_round(x, base = 0.05)) #bo karaty w oryginalnych danych są co .05

winsorize(model_data['price'], limits = [0.025, 0.1], inplace = True) # cutoff - .05
winsorize(model_data['carat'], limits = [0.05, 0.05], inplace = True) #, 'x dimension', 'y dimension', 'z dimension',
winsorize(model_data['x dimension'], limits = [0.05, 0.05], inplace = True)
winsorize(model_data['y dimension'], limits = [0.05, 0.05], inplace = True)
winsorize(model_data['z dimension'], limits = [0.05, 0.05], inplace = True)

dummies_str = pd.get_dummies(model_data.iloc[:,[1,2,3]], drop_first = True).astype('uint8')
model_data_wdummies = model_data.drop(['clarity','color','cut'],axis = 1)
model_data_wdummies = model_data_wdummies.join(dummies_str)
model_data_wdummies.columns = model_data_wdummies.columns.str.replace(" ", "")
model_data_wdummies.info()
#backward ols regression - repeat until pvalue <= .1 is satisfied


def lm_algorithm(data):
    r2 = []
    y = data['price']
    to_drop = ['price']
    plots_reg = []
    y_predictions = []
    i = 1
    no_dropped_columns = []
    while True:
        X = data.drop(to_drop, axis = 1)
        model = smf.ols(formula = "y ~ X", data = data).fit()
        least_important_value = enumerate(model.pvalues)
        maxes = [[value, index] for value, index in least_important_value]
        maxes = np.array(maxes)
        max_row = max(maxes, key=lambda x: x[1])
        max_index = max_row[0].astype('int')
        to_drop.append(X.columns[max_index - 1])
        no_dropped_columns.append([X.columns])
        r2.append(model.rsquared)
        print(model.summary(),to_drop)
        predictions = model.predict(X)
        residuals = y.values - predictions
        y_predictions.append(predictions)
        plt.figure(figsize=(10, 6))
        sns.regplot(x = predictions, y=model_data_wdummies.price,
            scatter_kws={'s': 50}, line_kws={'color': 'red', 'label': 'Regresja liniowa'})
        plots_reg.append(plt.gcf())
        plt.close()
        i += 1
        if model.pvalues.values.max() <= 0.10:
            break
    return r2, plots_reg, no_dropped_columns, y_predictions, i
r2_list, plots_reg, no_dropped_columns, y_pred_vals, i = lm_algorithm(model_data_wdummies)


# messy_data.iloc[:, 8:10] = messy_data.iloc[:, 8:10].apply(lambda x: x.astype(int))


na_count = messy_data.isna().to_numpy().sum()
na_pct = na_count / (messy_data.shape[0] * messy_data.shape[1])
#
na_count_carat = messy_data.iloc[:, 0].isna().to_numpy().sum()
na_pct_carat = na_count_carat / messy_data.shape[0]

counts_carat = (
    messy_data.iloc[:, 0].value_counts(dropna=False).sort_index()
)  # dropna = False - wyrzucamy bo i tak px nie obsługuje
max_counts_carat = messy_data.iloc[:, 0].value_counts(dropna=False).values[1]

fig_carat = px.histogram(
    messy_data,
    x="carat",
    nbins=len(range(min(counts_carat.values), max(counts_carat.values))),
).update_layout(bargap=0.01)
fig_carat.update_xaxes(
    autorange=True,
    # minor_ticks = "outside",
    dtick=0.05,
)
fig_carat.update_yaxes(autorangeoptions_maxallowed=max_counts_carat + 1)

list_messy_data_histograms.append(fig_carat)
list_napctg.append(na_pct_carat)

#
na_count_clarity = messy_data.iloc[:, 1].isna().to_numpy().sum()
na_pct_clarity = na_count_clarity / messy_data.shape[0]

counts_clarity = (
    messy_data.iloc[:, 1].value_counts(dropna=False).sort_index()
)  # dropna = False - wyrzucamy bo i tak px nie obsługuje
max_counts_clarity = messy_data.iloc[:, 1].value_counts(dropna=False).values[1].max()

fig_clarity = px.histogram(
    messy_data,
    x="clarity",
    nbins=len(range(min(counts_clarity.values), max(counts_clarity.values))),
).update_layout(bargap=0.01)

fig_clarity.update_yaxes(autorangeoptions_maxallowed=max_counts_clarity + 20)

list_messy_data_histograms.append(fig_clarity)
list_napctg.append(na_pct_clarity)
#
na_count_color = messy_data.iloc[:, 2].isna().to_numpy().sum()
na_pct_color = na_count_color / messy_data.shape[0]

counts_color = (
    messy_data.iloc[:, 2].value_counts(dropna=False).sort_index()
)  # dropna = False - wyrzucamy bo i tak px nie obsługuje
max_counts_color = messy_data.iloc[:, 2].value_counts(dropna=False).values[1].max()

fig_color = px.histogram(
    messy_data,
    x="color",
    nbins=len(range(min(counts_color.values), max(counts_color.values))),
).update_layout(bargap=0.01)

fig_color.update_yaxes(autorangeoptions_maxallowed=max_counts_color + 10)

list_messy_data_histograms.append(fig_color)
list_napctg.append(na_pct_color)
#
na_count_cut = messy_data.iloc[:, 3].isna().to_numpy().sum()
na_pct_cut = na_count_cut / messy_data.shape[0]

counts_cut = (
    messy_data.iloc[:, 3].value_counts(dropna=False).sort_index()
)  # dropna = False - wyrzucamy bo i tak px nie obsługuje
max_counts_cut = messy_data.iloc[:, 3].value_counts(dropna=False).values[1].max()

fig_cut = px.histogram(
    messy_data,
    x="cut",
    nbins=len(range(min(counts_cut.values), max(counts_cut.values))),
).update_layout(bargap=0.01)

fig_cut.update_yaxes(autorangeoptions_maxallowed=max_counts_cut + 15)

list_messy_data_histograms.append(fig_cut)
list_napctg.append(na_pct_cut)
#
na_count_xdim = messy_data.iloc[:, 4].isna().to_numpy().sum()
na_pct_xdim = na_count_xdim / messy_data.shape[0]

counts_xdim = (
    messy_data.iloc[:, 4].value_counts(dropna=True).sort_index()
)  # dropna = False - wyrzucamy bo i tak px nie obsługuje
max_counts_xdim = messy_data.iloc[:, 4].value_counts(dropna=False).values[1].max()

fig_xdim = px.histogram(
    messy_data,
    x="x dimension",
    nbins=len(range(min(counts_xdim.values), max(counts_xdim.values))),
).update_layout(bargap=0.01)

fig_xdim.update_yaxes(maxallowed=max_counts_xdim *5)

list_messy_data_histograms.append(fig_xdim)
list_napctg.append(na_pct_xdim)
#
na_count_ydim = messy_data.iloc[:, 5].isna().to_numpy().sum()
na_pct_ydim = na_count_carat / messy_data.shape[0]

counts_ydim = (
    messy_data.iloc[:, 5].value_counts(dropna=False).sort_index()
)  # dropna = False - wyrzucamy bo i tak px nie obsługuje
max_counts_ydim = messy_data.iloc[:, 5].value_counts(dropna=False).values[1].max()

fig_ydim = px.histogram(
    messy_data,
    x="y dimension",
    nbins=len(range(min(counts_carat.values), max(counts_carat.values))),
).update_layout(bargap=0.01)

fig_ydim.update_yaxes(autorangeoptions_maxallowed=max_counts_ydim + 7)

list_messy_data_histograms.append(fig_ydim)
list_napctg.append(na_pct_ydim)
#
na_count_zdim = messy_data.iloc[:, 6].isna().to_numpy().sum()
na_pct_zdim = na_count_zdim / messy_data.shape[0]

counts_zdim = (
    messy_data.iloc[:, 6].value_counts(dropna=False).sort_index()
)  # dropna = False - wyrzucamy bo i tak px nie obsługuje
max_counts_zdim = messy_data.iloc[:, 6].value_counts(dropna=False).values[1].max()

fig_zdim = px.histogram(
    messy_data,
    x="z dimension",
    nbins=len(range(min(counts_zdim.values), max(counts_zdim.values))),
).update_layout(bargap=0.01)

fig_zdim.update_yaxes(maxallowed=max_counts_zdim + 3)

list_messy_data_histograms.append(fig_zdim)
list_napctg.append(na_pct_zdim)
#
na_count_depth = messy_data.iloc[:, 7].isna().to_numpy().sum()
na_pct_depth = na_count_depth / messy_data.shape[0]

counts_depth = (
    messy_data.iloc[:, 7].value_counts(dropna=False).sort_index()
)  # dropna = False - wyrzucamy bo i tak px nie obsługuje
max_counts_depth = messy_data.iloc[:, 7].value_counts(dropna=False).values[1].max()

fig_depth = px.histogram(
    messy_data,
    x="depth",
    nbins=len(range(min(counts_depth.values), max(counts_depth.values))),
).update_layout(bargap=0.01)

fig_depth.update_yaxes(maxallowed=max_counts_depth)

list_messy_data_histograms.append(fig_depth)
list_napctg.append(na_pct_depth)
#
na_count_table = messy_data.iloc[:, 8].isna().to_numpy().sum()
na_pct_table = na_count_table / messy_data.shape[0]

counts_table = (
    messy_data.iloc[:, 8].value_counts(dropna=False).sort_index()
)  # dropna = False - wyrzucamy bo i tak px nie obsługuje
max_counts_table = messy_data.iloc[:, 8].value_counts(dropna=False).values[1].max()

fig_table = px.histogram(
    messy_data,
    x="table",
    nbins=len(range(min(counts_table.values), max(counts_table.values))),
).update_layout(bargap=0.01)

fig_table.update_yaxes(autorangeoptions_maxallowed=max_counts_table + 1)

list_messy_data_histograms.append(fig_table)
list_napctg.append(na_pct_table)
#
na_count_price = messy_data.iloc[:, 9].isna().to_numpy().sum()
na_pct_price = na_count_price / messy_data.shape[0]

counts_price = (
    messy_data.iloc[:, 9].value_counts(dropna=False).sort_index()
)  # dropna = False - wyrzucamy bo i tak px nie obsługuje
max_counts_price = messy_data.iloc[:, 9].value_counts(dropna=False).values[1].max()

fig_price = px.histogram(
    messy_data,
    x="price",
    nbins=len(range((max(counts_price.values) / min(counts_price.values)).astype('int'))),
).update_layout(bargap=0.01)

fig_price.update_yaxes(autorangeoptions_maxallowed=max_counts_price*40)

list_messy_data_histograms.append(fig_price)
list_napctg.append(na_pct_price)


add_selectboxmain = st.sidebar.selectbox(
    'Wybierz opcje do wyświetlenia...',
    ('Wizualizacje zmiennych przed transformacją', 
     "Statystyki opisowe zmiennych przed transformacją", "Wykresy punktowe między zmiennymi objaśniającymi za zmienną objaśnianą price",
     'Mapa ciepła dla korelacji między zmiennymi', "Wykresy regresji liniowej budowanej metodą indukcji wstecznej")
)
st.sidebar.write("Wybrałeś opcję: ", add_selectboxmain)
if add_selectboxmain == "Statystyki opisowe zmiennych przed transformacją":
    messy_data_describe = messy_data.describe()
    st.write(messy_data_describe)

elif add_selectboxmain == "Wykresy punktowe między zmiennymi objaśniającymi za zmienną objaśnianą price":
    col1, col2 = st.columns([0.25, 0.75])
    with col1:
        x_axis = st.selectbox('Zmienna objaśniana: ', messy_data.columns)
    with col2:
        scatter, ax = plt.subplots()
        sns.scatterplot(x = x_axis, y = messy_data['price'], data = messy_data)
        st.write(scatter)

elif add_selectboxmain == "Mapa ciepła dla korelacji między zmiennymi":
    fig, ax = plt.subplots()
    dummies_str = pd.get_dummies(messy_data.iloc[:,[1,2,3]])
    messy_data_wdummies = messy_data.drop(['clarity','color','cut'],axis = 1)
    messy_data_wdummies = messy_data_wdummies.join(dummies_str)
    sns.heatmap(messy_data_wdummies.corr(), ax=ax)
    st.write(fig)
elif add_selectboxmain == "Wykresy regresji liniowej budowanej metodą indukcji wstecznej":
    add_slider = st.slider(
        "Wybierz iterację: ",
        min_value = 1, max_value = i - 1, step = 1
    ) 
    st.write('Wykres regresji oraz jej reszt dla zestawu zmiennych objaśnianych: ', no_dropped_columns[add_slider - 1])
    st.write(plots_reg[add_slider - 1])


elif add_selectboxmain == "Wizualizacje zmiennych przed transformacją":
    add_selectbox = st.selectbox(
        "Wybierz, dla której zmiennej przed obróbką danych wyświetlic histogram.",
        ('BRAK','carat', 'clarity','color', 'cut', 'x dimension', 'y dimension', 'z dimension', 'depth', 'table', 'price')
    )
    if add_selectbox == 'BRAK':
        st.write('Nie wybrałeś/aś żadnej zmienej')
    if add_selectbox != 'BRAK':
        st.write('Wybrałeś/aś zmienną: ', add_selectbox)

    match add_selectbox:
        case 'carat':
            st.title(f':green[{messy_data.columns[0]}] frequency histogram')
            st.title(f':red[NA:] {list_napctg[0] * 100}%')
            st.plotly_chart(list_messy_data_histograms[0])
        case 'clarity':
            st.title(f':green[{messy_data.columns[1]}] frequency histogram')
            st.title(f':red[NA:] {list_napctg[1] * 100}%')
            st.plotly_chart(list_messy_data_histograms[1])
        case "color":
            st.title(f':green[{messy_data.columns[2]}] frequency histogram')
            st.title(f':red[NA:] {list_napctg[2] * 100}%')
            st.plotly_chart(list_messy_data_histograms[2])
        
        case "cut":
            st.title(f':green[{messy_data.columns[3]}] frequency histogram')
            st.title(f':red[NA:] {list_napctg[3] * 100}%')
            st.plotly_chart(list_messy_data_histograms[3])
        case "x dimension":
            st.title(f':green[{messy_data.columns[4]}] frequency histogram')
            st.title(f':red[NA:] {list_napctg[4] * 100}%')
            st.plotly_chart(list_messy_data_histograms[4])
        case "y dimension":
            st.title(f':green[{messy_data.columns[5]}] frequency histogram')
            st.title(f':red[NA:] {list_napctg[5] * 100}%')
            st.plotly_chart(list_messy_data_histograms[5])
        case "z dimension":
            st.title(f':green[{messy_data.columns[6]}] frequency histogram')
            st.title(f':red[NA:] {list_napctg[6] * 100}%')
            st.plotly_chart(list_messy_data_histograms[6])
        case "depth":
            st.title(f':green[{messy_data.columns[7]}] frequency histogram')
            st.title(f':red[NA:] {list_napctg[7] * 100}%')
            st.plotly_chart(list_messy_data_histograms[7])
        case "table":
            st.title(f':green[{messy_data.columns[8]}] frequency histogram')
            st.title(f':red[NA:] {list_napctg[8] * 100}%')
            st.plotly_chart(list_messy_data_histograms[8])
        case "price":
            st.title(f':green[{messy_data.columns[9]}] frequency histogram')
            st.title(f':red[NA:] {list_napctg[9] * 100}%')
            st.plotly_chart(list_messy_data_histograms[9])


